In [2]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import datasets
from catboost import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info

sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)

In [3]:
%cd "C:\Academics\Machine Learning\Kaggle predict future sales\competitive-data-science-predict-future-sales"

C:\Academics\Machine Learning\Kaggle predict future sales\competitive-data-science-predict-future-sales


In [4]:
data = pd.read_pickle('data.pkl')
test  = pd.read_csv('test.csv').set_index('ID')

In [5]:
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    #'date_shop_type_avg_item_cnt_lag_1',
    #'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    #'date_type_avg_item_cnt_lag_1',
    #'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]


In [192]:
data

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_shop_subtype_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,date_type_avg_item_cnt_lag_1,date_subtype_avg_item_cnt_lag_1,delta_price_lag,delta_revenue_lag_1,month,days,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
4488710,12,2,27,0.0,0,19,5,10,0.0,0.0,0.0,0.0,1.0,0.411377,0.086975,0.044434,0.130493,0.065247,0.155518,0.148071,0.100647,0.089050,0.096008,0.141235,1.082031,0.955566,0.808594,0.935059,0.148071,0.0,1.034180,1.075195,-0.282715,1.211914,0,31,1,1,12,12
4488711,12,2,30,0.0,0,40,11,4,0.0,0.0,0.0,0.0,0.0,0.411377,1.021484,1.022461,0.521973,0.891113,0.000000,0.148071,0.100647,0.089050,0.096008,0.000000,0.291504,0.046234,0.051727,0.046234,0.148071,0.0,0.270020,0.291504,-0.483398,1.211914,0,31,1,1,11,11
4488712,12,2,31,0.0,0,37,11,1,0.0,0.0,0.0,0.0,0.0,0.411377,0.543457,0.600098,0.543457,0.304443,0.000000,0.148071,0.100647,0.089050,0.096008,0.000000,0.232788,0.059448,0.051727,0.064697,0.148071,0.0,0.270020,0.234009,-0.137451,1.211914,0,31,1,1,11,11
4488713,12,2,32,1.0,0,40,11,4,0.0,0.0,0.0,0.0,0.0,0.411377,1.934570,1.799805,1.260742,1.891602,5.378906,0.148071,0.100647,0.089050,0.096008,0.141235,0.291504,0.046234,0.051727,0.046234,0.148071,0.0,0.270020,0.291504,-0.407227,1.211914,0,31,-1,1,12,12
4488714,12,2,33,1.0,0,37,11,1,1.0,2.0,0.0,0.0,1.0,0.411377,0.913086,0.333252,0.717285,1.000000,1.355469,0.148071,0.100647,0.089050,0.096008,0.141235,0.232788,0.059448,0.051727,0.064697,0.148071,1.0,0.270020,0.234009,-0.225464,1.211914,0,31,1,1,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127999,34,45,18454,0.0,20,55,13,2,1.0,0.0,0.0,0.0,0.0,0.258545,0.045441,0.023254,0.071411,0.590820,0.000000,0.126709,0.128784,0.139038,0.137573,0.000000,0.196899,0.126831,0.089294,0.126831,0.135376,0.5,0.146973,0.196899,-0.475098,-0.290039,10,30,1,-1,11,11
11128000,34,45,16188,0.0,20,64,14,42,0.0,0.0,0.0,0.0,0.0,0.258545,0.022720,0.069763,0.000000,0.000000,0.000000,0.126709,0.128784,0.000000,0.000000,0.000000,0.155884,0.094482,0.106201,0.094482,0.135376,0.0,0.237305,0.155884,0.081116,-0.290039,10,30,-1,-1,2,2
11128001,34,45,15757,0.0,20,55,13,2,0.0,0.0,0.0,0.0,0.0,0.258545,0.113647,0.069763,0.095215,0.250000,0.180054,0.126709,0.128784,0.139038,0.137573,0.181274,0.196899,0.126831,0.089294,0.126831,0.135376,0.0,0.146973,0.196899,0.155884,-0.290039,10,30,1,-1,34,34
11128002,34,45,19648,0.0,20,40,11,4,0.0,0.0,0.0,0.0,0.0,0.258545,0.045441,0.069763,0.166626,0.090881,0.000000,0.126709,0.128784,0.139038,0.137573,0.000000,0.220825,0.083740,0.097046,0.083740,0.135376,0.0,0.224243,0.220825,-0.091736,-0.290039,10,30,-1,-1,11,11


In [6]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [196]:
Y_train = Y_train.astype(np.float32)
Y_valid = Y_valid.astype(np.float32)


In [97]:
X_train

,date_block_num,shop_id,item_id,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,delta_price_lag,month,days,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
4488710,12,2,27,0,19,5,10,0.0,0.0,0.0,0.0,1.0,0.411377,0.086975,0.044434,0.130493,0.065247,0.155518,0.148071,0.100647,0.089050,0.096008,0.141235,1.082031,0.955566,0.148071,0.0,-0.282715,0,31,1,1,12,12
4488711,12,2,30,0,40,11,4,0.0,0.0,0.0,0.0,0.0,0.411377,1.021484,1.022461,0.521973,0.891113,0.000000,0.148071,0.100647,0.089050,0.096008,0.000000,0.291504,0.046234,0.148071,0.0,-0.483398,0,31,1,1,11,11
4488712,12,2,31,0,37,11,1,0.0,0.0,0.0,0.0,0.0,0.411377,0.543457,0.600098,0.543457,0.304443,0.000000,0.148071,0.100647,0.089050,0.096008,0.000000,0.232788,0.059448,0.148071,0.0,-0.137451,0,31,1,1,11,11
4488713,12,2,32,0,40,11,4,0.0,0.0,0.0,0.0,0.0,0.411377,1.934570,1.799805,1.260742,1.891602,5.378906,0.148071,0.100647,0.089050,0.096008,0.141235,0.291504,0.046234,0.148071,0.0,-0.407227,0,31,-1,1,12,12
4488714,12,2,33,0,37,11,1,1.0,2.0,0.0,0.0,1.0,0.411377,0.913086,0.333252,0.717285,1.000000,1.355469,0.148071,0.100647,0.089050,0.096008,0.141235,0.232788,0.059448,0.148071,1.0,-0.225464,0,31,1,1,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10675627,32,59,22162,30,40,11,4,1.0,0.0,0.0,4.0,0.0,0.288086,0.333252,0.627930,0.511719,3.826172,0.000000,0.218506,0.184692,0.172119,0.186768,0.000000,0.242188,0.111633,0.218506,1.0,-0.085815,8,30,1,-1,14,14
10675628,32,59,22163,30,40,11,4,0.0,0.0,0.0,0.0,0.0,0.288086,0.690430,0.000000,0.000000,0.000000,0.000000,0.218506,0.000000,0.000000,0.000000,0.000000,0.242188,0.111633,0.218506,0.0,-0.008354,8,30,-1,-1,1,1
10675629,32,59,22164,30,37,11,1,0.0,1.0,0.0,1.0,0.0,0.288086,0.476074,0.279053,0.162842,2.000000,0.000000,0.218506,0.184692,0.172119,0.186768,0.000000,0.192017,0.122009,0.218506,0.0,-0.114929,8,30,1,-1,7,7
10675630,32,59,22166,30,54,12,61,0.0,0.0,0.0,0.0,0.0,0.288086,0.261963,0.186035,0.232544,0.260986,0.260010,0.218506,0.184692,0.172119,0.186768,0.175781,0.143433,0.000000,0.218506,0.0,0.000000,8,30,-1,-1,23,23


In [59]:
Y_train.unique()

array([ 0.,  1.,  2.,  7.,  9.,  3.,  4., 10.,  6.,  5., 20., 13., 11.,
       12.,  8., 16., 18., 19., 14., 15., 17.], dtype=float32)

In [62]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=20,
    random_seed=0,
    learning_rate=0.1
)
model.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 1.6458976	test: 1.6157919	best: 1.6157919 (0)	total: 51.2s	remaining: 16m 12s
1:	learn: 1.4122718	test: 1.3857932	best: 1.3857932 (1)	total: 1m 51s	remaining: 16m 41s
2:	learn: 1.2503455	test: 1.2256970	best: 1.2256970 (2)	total: 2m 55s	remaining: 16m 36s
3:	learn: 1.1278326	test: 1.1046068	best: 1.1046068 (3)	total: 3m 59s	remaining: 15m 57s
4:	learn: 1.0308950	test: 1.0087205	best: 1.0087205 (4)	total: 5m	remaining: 15m 1s
5:	learn: 0.9525987	test: 0.9308526	best: 0.9308526 (5)	total: 6m 2s	remaining: 14m 6s
6:	learn: 0.8874610	test: 0.8660128	best: 0.8660128 (6)	total: 7m 10s	remaining: 13m 19s
7:	learn: 0.8324204	test: 0.8114800	best: 0.8114800 (7)	total: 8m 24s	remaining: 12m 36s
8:	learn: 0.7856876	test: 0.7653156	best: 0.7653156 (8)	total: 9m 33s	remaining: 11m 41s
9:	learn: 0.7457659	test: 0.7256759	best: 0.7256759 (9)	total: 10m 39s	remaining: 10m 39s
10:	learn: 0.7111031	test: 0.6916185	best: 0.6916185 (10)	total: 11m 44s	remaining: 9m 36s
11:	learn: 0.6809134	test:

In [68]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)


Exception: Data must be 1-dimensional

In [ ]:
Y_test1=[]
for i in Y_test:
    Y_test1.append(i[0])
Y_test1

In [79]:

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test1
})
submission.to_csv('catboost_submission_cat.csv', index=False)

# save predictions for an ensemble
pickle.dump(Y_pred, open('catboost_train.pickle', 'wb'))
pickle.dump(Y_test, open('catboost_test.pickle', 'wb'))

# CATBOOST REGRESSOR

In [100]:
cat_features=[0,1,2,3,4,5,6,33,32,31,30,29,28]

In [108]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(
    iterations=100,loss_function='RMSE',

    random_seed=0,
    learning_rate=0.05
)
model.fit(
    X_train, Y_train,cat_features=cat_features,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 1.1670518	test: 1.1218166	best: 1.1218166 (0)	total: 7.92s	remaining: 13m 4s
1:	learn: 1.1474854	test: 1.1084251	best: 1.1084251 (1)	total: 11.7s	remaining: 9m 31s
2:	learn: 1.1293758	test: 1.0961953	best: 1.0961953 (2)	total: 14.5s	remaining: 7m 49s
3:	learn: 1.1126215	test: 1.0852372	best: 1.0852372 (3)	total: 18.5s	remaining: 7m 24s
4:	learn: 1.0972989	test: 1.0751861	best: 1.0751861 (4)	total: 21.4s	remaining: 6m 46s
5:	learn: 1.0829870	test: 1.0649928	best: 1.0649928 (5)	total: 24.4s	remaining: 6m 22s
6:	learn: 1.0697527	test: 1.0561542	best: 1.0561542 (6)	total: 27.2s	remaining: 6m
7:	learn: 1.0575795	test: 1.0483625	best: 1.0483625 (7)	total: 29.8s	remaining: 5m 43s
8:	learn: 1.0461449	test: 1.0407264	best: 1.0407264 (8)	total: 32.5s	remaining: 5m 28s
9:	learn: 1.0356566	test: 1.0341975	best: 1.0341975 (9)	total: 35.1s	remaining: 5m 15s
10:	learn: 1.0259614	test: 1.0267153	best: 1.0267153 (10)	total: 37.8s	remaining: 5m 5s
11:	learn: 1.0170752	test: 1.0208335	best: 1.0

In [85]:
from catboost import CatBoostRegressor
model_1 = CatBoostRegressor(
    iterations=1000,loss_function='RMSE',

    random_seed=0,
    learning_rate=0.05
)
model_1.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 1.1670516	test: 1.1216940	best: 1.1216940 (0)	total: 775ms	remaining: 12m 54s
1:	learn: 1.1474592	test: 1.1081586	best: 1.1081586 (1)	total: 1.52s	remaining: 12m 41s
2:	learn: 1.1293111	test: 1.0960590	best: 1.0960590 (2)	total: 2.36s	remaining: 13m 5s
3:	learn: 1.1127406	test: 1.0846140	best: 1.0846140 (3)	total: 3.12s	remaining: 12m 56s
4:	learn: 1.0970211	test: 1.0741239	best: 1.0741239 (4)	total: 3.94s	remaining: 13m 3s
5:	learn: 1.0823268	test: 1.0640280	best: 1.0640280 (5)	total: 4.76s	remaining: 13m 9s
6:	learn: 1.0690198	test: 1.0549161	best: 1.0549161 (6)	total: 5.42s	remaining: 12m 48s
7:	learn: 1.0563389	test: 1.0459030	best: 1.0459030 (7)	total: 6.14s	remaining: 12m 41s
8:	learn: 1.0447933	test: 1.0384335	best: 1.0384335 (8)	total: 6.81s	remaining: 12m 30s
9:	learn: 1.0341501	test: 1.0316988	best: 1.0316988 (9)	total: 7.48s	remaining: 12m 20s
10:	learn: 1.0239814	test: 1.0247725	best: 1.0247725 (10)	total: 8.18s	remaining: 12m 15s
11:	learn: 1.0148801	test: 1.0194

In [86]:
Y_pred = model_1.predict(X_valid).clip(0, 20)
Y_test = model_1.predict(X_test).clip(0, 20)


In [167]:
Y_test

array([0.7218, 0.4263, 1.2841, ..., 0.0527, 0.026 , 0.0378])

In [87]:

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('catboost_submission_cat_regressor.csv', index=False)

# save predictions for an ensemble
pickle.dump(Y_pred, open('catboost_train.pickle', 'wb'))
pickle.dump(Y_test, open('catboost_test.pickle', 'wb'))

In [27]:
from catboost import CatBoostRegressor
model_1 = CatBoostRegressor(
    iterations=600,loss_function='RMSE',

    random_seed=1,
    learning_rate=0.05
)
model_1.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

TypeError: No matching signature found

In [105]:
from catboost import CatBoostRegressor
model_2 = CatBoostRegressor(
    iterations=600,loss_function='RMSE',

    random_seed=2,
    learning_rate=0.05
)
model_2.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model_2.is_fitted()))
print('Model params:')
print(model_2.get_params())

0:	learn: 1.1670414	test: 1.1217626	best: 1.1217626 (0)	total: 896ms	remaining: 8m 56s
1:	learn: 1.1474227	test: 1.1082118	best: 1.1082118 (1)	total: 1.7s	remaining: 8m 28s
2:	learn: 1.1292202	test: 1.0959739	best: 1.0959739 (2)	total: 2.49s	remaining: 8m 16s
3:	learn: 1.1123876	test: 1.0845528	best: 1.0845528 (3)	total: 3.23s	remaining: 8m
4:	learn: 1.0968216	test: 1.0746174	best: 1.0746174 (4)	total: 3.86s	remaining: 7m 38s
5:	learn: 1.0823511	test: 1.0649010	best: 1.0649010 (5)	total: 4.6s	remaining: 7m 35s
6:	learn: 1.0691533	test: 1.0560777	best: 1.0560777 (6)	total: 5.27s	remaining: 7m 26s
7:	learn: 1.0567357	test: 1.0479679	best: 1.0479679 (7)	total: 6.01s	remaining: 7m 24s
8:	learn: 1.0451812	test: 1.0403573	best: 1.0403573 (8)	total: 6.72s	remaining: 7m 21s
9:	learn: 1.0341280	test: 1.0324773	best: 1.0324773 (9)	total: 7.46s	remaining: 7m 20s
10:	learn: 1.0242995	test: 1.0265541	best: 1.0265541 (10)	total: 8.18s	remaining: 7m 18s
11:	learn: 1.0151743	test: 1.0205840	best: 1.02

In [106]:
from catboost import CatBoostRegressor
model_3 = CatBoostRegressor(
    iterations=600,loss_function='RMSE',

    random_seed=3,
    learning_rate=0.05
)
model_3.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model_3.is_fitted()))
print('Model params:')
print(model_3.get_params())

0:	learn: 1.1670482	test: 1.1217498	best: 1.1217498 (0)	total: 685ms	remaining: 6m 50s
1:	learn: 1.1473934	test: 1.1086266	best: 1.1086266 (1)	total: 1.45s	remaining: 7m 13s
2:	learn: 1.1293211	test: 1.0962531	best: 1.0962531 (2)	total: 2.17s	remaining: 7m 12s
3:	learn: 1.1123275	test: 1.0843561	best: 1.0843561 (3)	total: 2.92s	remaining: 7m 14s
4:	learn: 1.0965579	test: 1.0732180	best: 1.0732180 (4)	total: 3.65s	remaining: 7m 14s
5:	learn: 1.0820667	test: 1.0637593	best: 1.0637593 (5)	total: 4.36s	remaining: 7m 11s
6:	learn: 1.0687998	test: 1.0549946	best: 1.0549946 (6)	total: 5.05s	remaining: 7m 8s
7:	learn: 1.0562064	test: 1.0463630	best: 1.0463630 (7)	total: 5.73s	remaining: 7m 4s
8:	learn: 1.0447822	test: 1.0386410	best: 1.0386410 (8)	total: 6.4s	remaining: 7m
9:	learn: 1.0338523	test: 1.0310214	best: 1.0310214 (9)	total: 7.07s	remaining: 6m 57s
10:	learn: 1.0240167	test: 1.0248097	best: 1.0248097 (10)	total: 7.72s	remaining: 6m 53s
11:	learn: 1.0149384	test: 1.0193110	best: 1.019

In [107]:
from catboost import CatBoostRegressor
model_10 = CatBoostRegressor(
    iterations=600,loss_function='RMSE',

    random_seed=10,
    learning_rate=0.05
)
model_10.fit(
    X_train, Y_train,
    eval_set=(X_valid, Y_valid),
    
)
print('Model is fitted: ' + str(model_10.is_fitted()))
print('Model params:')
print(model_10.get_params())

0:	learn: 1.1670582	test: 1.1215669	best: 1.1215669 (0)	total: 743ms	remaining: 7m 24s
1:	learn: 1.1474743	test: 1.1082667	best: 1.1082667 (1)	total: 1.48s	remaining: 7m 23s
2:	learn: 1.1293106	test: 1.0958675	best: 1.0958675 (2)	total: 2.16s	remaining: 7m 9s
3:	learn: 1.1125624	test: 1.0849752	best: 1.0849752 (3)	total: 2.88s	remaining: 7m 8s
4:	learn: 1.0971192	test: 1.0746673	best: 1.0746673 (4)	total: 3.63s	remaining: 7m 12s
5:	learn: 1.0825567	test: 1.0645626	best: 1.0645626 (5)	total: 4.32s	remaining: 7m 7s
6:	learn: 1.0687461	test: 1.0550449	best: 1.0550449 (6)	total: 5.03s	remaining: 7m 5s
7:	learn: 1.0564342	test: 1.0473302	best: 1.0473302 (7)	total: 5.65s	remaining: 6m 58s
8:	learn: 1.0448103	test: 1.0393261	best: 1.0393261 (8)	total: 6.4s	remaining: 7m
9:	learn: 1.0339782	test: 1.0319775	best: 1.0319775 (9)	total: 7.12s	remaining: 7m
10:	learn: 1.0240963	test: 1.0254271	best: 1.0254271 (10)	total: 7.77s	remaining: 6m 56s
11:	learn: 1.0146964	test: 1.0189134	best: 1.0189134 (

Add results of xgb as ensembles and perform catboost

In [169]:
y_xgb_pred=pd.read_pickle('xgb_train.pickle')
y_xgb_test=pd.read_pickle('xgb_test.pickle')

In [172]:
y_xgb_pred

array([0.0251, 0.1029, 0.1579, ..., 0.2052, 0.0132, 0.1522], dtype=float32)

In [149]:
X_valid.shape

(238172, 34)

In [161]:
X_valid1=X_valid.reset_index()
del X_valid1['index']
X_valid1.shape

(238172, 34)

In [166]:
new_train=X_valid1
new_train['y_xgb_pred']=y_xgb_pred
new_train

,date_block_num,shop_id,item_id,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,delta_price_lag,month,days,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale,y_xgb_pred
0,33,2,30,0,40,11,4,0.0,0.0,0.0,0.0,0.0,0.263672,0.069763,0.142822,0.093018,0.085083,0.250000,0.157959,0.184448,0.143311,0.125122,0.138916,0.200928,0.058746,0.157959,0.0,-0.632324,9,31,1,1,32,32,0.025101
1,33,2,31,0,37,11,1,0.0,0.0,0.0,0.0,0.0,0.263672,0.209351,1.261719,0.139526,0.276611,0.153809,0.157959,0.184448,0.143311,0.125122,0.138916,0.201904,0.117310,0.157959,0.0,-0.663574,9,31,1,1,32,32,0.102911
2,33,2,32,0,40,11,4,0.0,1.0,0.0,0.0,2.0,0.263672,0.441895,0.714355,0.488281,0.425537,0.711426,0.157959,0.184448,0.143311,0.125122,0.138916,0.200928,0.058746,0.157959,0.0,-0.403076,9,31,1,1,33,33,0.157920
3,33,2,33,0,37,11,1,1.0,0.0,1.0,1.0,0.0,0.263672,0.372070,0.333252,0.348877,0.276611,0.269287,0.157959,0.184448,0.143311,0.125122,0.138916,0.201904,0.117310,0.157959,1.0,-0.221924,9,31,1,1,33,33,0.286821
4,33,2,40,0,57,13,8,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.021271,0.038452,0.000000,0.000000,0.000000,0.125122,0.138916,0.000000,0.000000,0.000000,0.0,0.000000,9,31,-1,6,33,33,0.006808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238167,33,59,22162,30,40,11,4,0.0,1.0,0.0,1.0,0.0,0.263672,0.162842,0.333252,0.627930,1.659180,0.000000,0.179688,0.218506,0.184692,0.167480,0.000000,0.200928,0.109863,0.179688,0.0,-0.085815,9,31,1,-1,15,15,0.169851
238168,33,59,22163,30,40,11,4,0.0,0.0,0.0,0.0,0.0,0.263672,0.465088,0.690430,0.000000,0.000000,0.000000,0.179688,0.218506,0.000000,0.000000,0.000000,0.200928,0.109863,0.179688,0.0,0.026581,9,31,-1,-1,2,2,0.213264
238169,33,59,22164,30,37,11,1,0.0,0.0,1.0,2.0,0.0,0.263672,0.209351,0.476074,0.279053,0.574707,0.000000,0.179688,0.218506,0.184692,0.167480,0.000000,0.201904,0.131958,0.179688,0.0,-0.230591,9,31,1,-1,8,8,0.205164
238170,33,59,22166,30,54,12,61,0.0,0.0,0.0,0.0,0.0,0.263672,0.116272,0.261963,0.186035,0.085083,0.230713,0.179688,0.218506,0.184692,0.167480,0.178589,0.150635,0.000000,0.179688,0.0,0.000000,9,31,-1,-1,24,24,0.013182


In [186]:
new_test=X_test
new_test['y_xgb_pred']=y_xgb_test
new_test

,date_block_num,shop_id,item_id,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,delta_price_lag,month,days,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale,y_xgb_test,y_xgb_pred
10913804,34,5,5037,3,19,5,10,0.0,1.0,3.0,1.0,1.0,0.258545,0.568359,2.511719,2.833984,1.977539,1.299805,0.190063,0.205933,0.245117,0.180054,0.206055,0.379150,0.513672,0.190063,0.0,-0.222046,10,30,1,1,14,14,0.521888,0.521888
10913805,34,5,5320,3,55,13,2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,10,30,-1,-1,0,0,0.347764,0.347764
10913806,34,5,5233,3,19,5,10,1.0,3.0,1.0,3.0,0.0,0.258545,0.954590,1.860352,3.572266,1.613281,0.000000,0.190063,0.205933,0.245117,0.180054,0.000000,0.379150,0.513672,0.190063,1.0,0.496826,10,30,1,1,7,7,1.123582,1.123582
10913807,34,5,5232,3,23,5,16,0.0,0.0,1.0,0.0,0.0,0.258545,0.636230,1.116211,1.547852,0.000000,0.000000,0.190063,0.205933,0.245117,0.000000,0.000000,0.337402,0.374268,0.190063,0.0,0.505371,10,30,1,1,3,3,0.485232,0.485232
10913808,34,5,5268,3,20,5,11,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,10,30,-1,-1,0,0,2.283453,2.283453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127999,34,45,18454,20,55,13,2,1.0,0.0,0.0,0.0,0.0,0.258545,0.045441,0.023254,0.071411,0.590820,0.000000,0.126709,0.128784,0.139038,0.137573,0.000000,0.196899,0.126831,0.135376,0.5,-0.475098,10,30,1,-1,11,11,0.087469,0.087469
11128000,34,45,16188,20,64,14,42,0.0,0.0,0.0,0.0,0.0,0.258545,0.022720,0.069763,0.000000,0.000000,0.000000,0.126709,0.128784,0.000000,0.000000,0.000000,0.155884,0.094482,0.135376,0.0,0.081116,10,30,-1,-1,2,2,0.028624,0.028624
11128001,34,45,15757,20,55,13,2,0.0,0.0,0.0,0.0,0.0,0.258545,0.113647,0.069763,0.095215,0.250000,0.180054,0.126709,0.128784,0.139038,0.137573,0.181274,0.196899,0.126831,0.135376,0.0,0.155884,10,30,1,-1,34,34,0.076241,0.076241
11128002,34,45,19648,20,40,11,4,0.0,0.0,0.0,0.0,0.0,0.258545,0.045441,0.069763,0.166626,0.090881,0.000000,0.126709,0.128784,0.139038,0.137573,0.000000,0.220825,0.083740,0.135376,0.0,-0.091736,10,30,-1,-1,11,11,0.017915,0.017915


In [191]:
from catboost import CatBoostRegressor
model_11 = CatBoostRegressor(
    iterations=1000,loss_function='RMSE',

    random_seed=0,
    learning_rate=0.01
)
model_11.fit(
    new_train, Y_valid
    
)
print('Model is fitted: ' + str(model_11.is_fitted()))
print('Model params:')
print(model_11.get_params())

0:	learn: 1.1321784	total: 30.3ms	remaining: 30.3s
1:	learn: 1.1282824	total: 56.2ms	remaining: 28s
2:	learn: 1.1244619	total: 80.9ms	remaining: 26.9s
3:	learn: 1.1206843	total: 106ms	remaining: 26.4s
4:	learn: 1.1169601	total: 129ms	remaining: 25.7s
5:	learn: 1.1133333	total: 151ms	remaining: 25s
6:	learn: 1.1096873	total: 177ms	remaining: 25.1s
7:	learn: 1.1061401	total: 206ms	remaining: 25.5s
8:	learn: 1.1026613	total: 229ms	remaining: 25.2s
9:	learn: 1.0991769	total: 256ms	remaining: 25.4s
10:	learn: 1.0957613	total: 290ms	remaining: 26.1s
11:	learn: 1.0924651	total: 317ms	remaining: 26.1s
12:	learn: 1.0892052	total: 344ms	remaining: 26.1s
13:	learn: 1.0860104	total: 368ms	remaining: 25.9s
14:	learn: 1.0827927	total: 394ms	remaining: 25.9s
15:	learn: 1.0796257	total: 421ms	remaining: 25.9s
16:	learn: 1.0764746	total: 450ms	remaining: 26s
17:	learn: 1.0734091	total: 478ms	remaining: 26.1s
18:	learn: 1.0704328	total: 502ms	remaining: 25.9s
19:	learn: 1.0675031	total: 522ms	remaining:

In [187]:
Y_test = model_11.predict(new_test).clip(0, 20)


In [188]:

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('catboost_submission_cat_regressor_xgb_ensembles.csv', index=False)

# save predictions for an ensemble
#pickle.dump(Y_pred, open('catboost_train.pickle', 'wb'))
#pickle.dump(Y_test, open('catboost_test.pickle', 'wb'))

# STACKNET

In [19]:
%cd "C:\Academics\Machine Learning\Kaggle predict future sales\competitive-data-science-predict-future-sales"

C:\Academics\Machine Learning\Kaggle predict future sales\competitive-data-science-predict-future-sales


In [18]:
import pystacknet


In [20]:
import gc
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

# Machine Learning
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


seed = 1234
np.random.seed(seed)

t_start = time.time()

In [21]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [31]:
Y_train = Y_train.astype(np.float32)
Y_valid = Y_valid.astype(np.float32)


In [32]:
Y_train.dtype

dtype('float32')

In [33]:
# Level 1 are the base models that take the training dataset as input
l1_clf1 = GradientBoostingRegressor(n_estimators=400,
                                    learning_rate=0.006,
                                    min_samples_leaf=10,
                                    max_depth=15, 
                                    max_features='sqrt', 
                                    subsample=0.85,
                                    random_state=seed)

l1_clf2 = LGBMRegressor(boosting_type='gbdt',
                        objective="binary",
                        metric="AUC",
                        boost_from_average="false",
                        learning_rate=0.0045,
                        num_leaves=491,
                        max_depth=20,
                        min_child_weight=0.035,
                        feature_fraction=0.38,
                        bagging_fraction=0.42,
                        min_data_in_leaf=100,
                        max_bin=255,
                        importance_type='split',
                        reg_alpha=0.4,
                        reg_lambda=0.65,
                        bagging_seed=seed,
                        random_state=seed,
                        verbosity=-1,
                        subsample=0.85,
                        colsample_bytree=0.8,
                        min_child_samples=79)

l1_clf3 = CatBoostRegressor(learning_rate=0.2,
                            bagging_temperature=0.1, 
                            l2_leaf_reg=30,
                            depth=12, 
                            max_bin=255,
                            iterations=100,
                            loss_function='Logloss',
                            objective='RMSE',
                            eval_metric="AUC",
                            bootstrap_type='Bayesian',
                            random_seed=seed,
                            early_stopping_rounds=10)

# Level 2 models will take predictions from level 1 models as input
# Remember to keep level 2 models smaller
# Basic models like Ridge Regression with large regularization or small random forests work well
l2_clf1 = RandomForestRegressor(n_estimators=250, 
                                max_depth=5, 
                                max_features='sqrt', 
                                random_state=seed)

In [23]:
# Specify model tree for StackNet
models = [[l1_clf1, l1_clf2, l1_clf3], # Level 1
          [l2_clf1]] # Level 2

In [25]:

FOLDS=4
METRIC="rmse"
RESTACKING=False
RETRAIN=True
SEED=12345
VERBOSE=1


In [34]:
from pystacknet.pystacknet import StackNetRegressor

# Specify parameters for stacked model and begin training
model=StackNetRegressor(models, metric=METRIC, folds=FOLDS,
    restacking=RESTACKING, use_retraining=RETRAIN, 
    random_state=SEED, verbose=VERBOSE)

# Fit the entire model tree
model.fit(X_train, Y_train)

====================== Start of Level 0 ======================
Input Dimensionality 34 at Level 0 
3 models included in Level 0 
0:	total: 2.93s	remaining: 4m 50s
1:	total: 5.76s	remaining: 4m 42s
2:	total: 8.45s	remaining: 4m 33s
3:	total: 11.2s	remaining: 4m 28s
4:	total: 13.9s	remaining: 4m 23s
5:	total: 16.6s	remaining: 4m 19s
6:	total: 19.3s	remaining: 4m 16s
7:	total: 22s	remaining: 4m 13s
8:	total: 24.6s	remaining: 4m 9s
9:	total: 27.3s	remaining: 4m 5s
10:	total: 30s	remaining: 4m 2s
11:	total: 32.8s	remaining: 4m
12:	total: 35.5s	remaining: 3m 57s
13:	total: 38.1s	remaining: 3m 54s
14:	total: 40.8s	remaining: 3m 51s
15:	total: 43.4s	remaining: 3m 48s
16:	total: 46.1s	remaining: 3m 45s
17:	total: 48.8s	remaining: 3m 42s
18:	total: 51.5s	remaining: 3m 39s
19:	total: 54.1s	remaining: 3m 36s
20:	total: 56.8s	remaining: 3m 33s
21:	total: 59.5s	remaining: 3m 30s
22:	total: 1m 2s	remaining: 3m 27s
23:	total: 1m 4s	remaining: 3m 24s
24:	total: 1m 7s	remaining: 3m 22s
25:	total: 1m 10s

In [35]:
preds1=model.predict(X_test)


====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 


In [58]:
Y_test=[]
for i in preds1:
    Y_test.append(i[0])
Y_test

[0.6590941373707717,
 0.5042953092155273,
 1.4577068107524405,
 0.6826956552465556,
 2.5043560978098243,
 0.7691571540846887,
 0.7460740209810166,
 0.14847187225296485,
 0.7412126582370835,
 0.6445473771041235,
 3.20525156112748,
 0.2461872843593182,
 0.14178511009739259,
 0.4769887698116677,
 1.8857160244659252,
 2.715682501632111,
 0.04899482986171957,
 0.16114628930434227,
 1.4948550575723722,
 0.31733064294339947,
 0.4384783751247527,
 0.3075163876315453,
 2.046999411201905,
 0.553739679864737,
 1.3914911883819345,
 0.7111044879302206,
 0.44704765994583645,
 0.4844736844360111,
 1.1722567008059193,
 4.357314335422429,
 0.769029639673342,
 0.6747050744297193,
 0.973321892421293,
 1.418332775505804,
 0.06129734445200211,
 0.06372669514456057,
 0.06409365406333291,
 1.1339072446966247,
 0.19073404885747036,
 0.151019141916266,
 0.7515797221389542,
 1.5233319057872698,
 3.2454942837241534,
 4.104653917394721,
 1.29978029510705,
 0.26501620994900754,
 1.1129584196893478,
 0.459056912361

In [59]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('stacknet_1.csv', index=False)

# Stacknet 2

In [51]:
from pystacknet.pystacknet import StackNetRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition.pca import PCA
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasRegressor




# Read data
def keras_model():
    # Here's a Deep Dumb MLP (DDMLP)
    model = Sequential()
    model.add(Dense(128, input_dim=34))
    model.add(Activation('relu'))
    #model.add(Dropout(0.15))
    model.add(Dense(128))
    model.add(Activation('relu'))
    #model.add(Dropout(0.15))
    model.add(Dense(1))
    model.add(Activation('linear'))

    # we'll use categorical xent for the loss, and RMSprop as the optimizer
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model



In [52]:

FOLDS=4
METRIC="rmse"
RESTACKING=True
RETRAIN=False
SEED=12345
VERBOSE=0

models=[
    
    #1ST level #
    [Ridge(alpha=0.001, normalize=True, random_state=1234),
    GradientBoostingRegressor(n_estimators=200,learning_rate=0.05,max_features=0.2,min_samples_leaf=20,max_depth=6,random_state=1),    
    ExtraTreesRegressor(n_estimators=200, max_features=0.5, max_depth=15, random_state=1234 ),
    MLPRegressor(hidden_layer_sizes=(100,50 ), activation="relu", solver="adam",alpha=0.01,batch_size=30, learning_rate="adaptive",learning_rate_init=0.001, power_t=0.5,max_iter=20, shuffle=True, random_state=1, tol=0.0001, momentum=0.9,validation_fraction=0.1,beta_1=0.1, beta_2=0.1, epsilon=0.1),
    KerasRegressor(build_fn=keras_model, epochs=10, batch_size=15, verbose=0),
    #PCA(n_components=1, random_state=1)
    
    ],
     
        #2ND level # 

        [ Ridge(alpha=0.001, normalize=True, random_state=1234)],    

]

#MODEL STATEMENT
model=StackNetRegressor(models, metric=METRIC, folds=FOLDS,
    restacking=RESTACKING, use_retraining=RETRAIN, 
    random_state=SEED, verbose=VERBOSE)


In [53]:
model.fit(X_train, Y_train)


C:\Users\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [54]:
preds2=model.predict(X_test)


In [60]:
Y_test=[]
for i in preds2:
    Y_test.append(i[0])
Y_test

[0.6334957266829,
 0.5016430811615646,
 1.2003923231397855,
 0.4740079741080723,
 4.6988442390781096,
 0.6577138775784148,
 0.7949938122343713,
 0.14780980728000648,
 0.7409593779563368,
 0.7108667635611774,
 3.1007553453997403,
 0.14506542866434505,
 0.20799331658330114,
 0.521205595859138,
 1.7830976075245173,
 2.7746160591564872,
 0.022687882112575622,
 0.2373840283277498,
 1.1757890513567129,
 0.2628415478850702,
 0.3645645663548604,
 0.2504547107321565,
 1.8716492488068877,
 0.4359475514776334,
 1.3783900876214652,
 0.7890739638139791,
 0.4441055265899424,
 0.6310452676807834,
 1.0357609610858813,
 5.224930439162429,
 0.7756704658134387,
 0.42741522584371344,
 0.9991702051258267,
 1.1422285593581571,
 -0.009335403557689928,
 0.012440143889567365,
 0.014516046839686415,
 1.1471632719799325,
 0.2527951064112227,
 0.15788967366865175,
 0.7432862261180322,
 1.539057435289815,
 3.629973827127531,
 4.405610608047057,
 1.1661387684231028,
 0.29838471613193934,
 1.2174422212960696,
 0.510

In [57]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('stacknet_2.csv', index=False)

# PCA

X_train,Y_train,X_test,X_valid,Y_valid

In [62]:
from sklearn.decomposition import PCA
pca=PCA(n_components=10)


In [63]:
pca.fit(X_train)


PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [64]:
X_train_pca=pca.transform(X_train)
X_valid_pca=pca.transform(X_valid)
X_test_pca=pca.transform(X_test)


In [66]:
from xgboost import XGBRegressor
from xgboost import plot_importance

In [67]:
ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train_pca, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train_pca, Y_train), (X_valid_pca, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

[20:54:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:1.06864	validation_1-rmse:1.06411
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.00074	validation_1-rmse:1.01667
[2]	validation_0-rmse:0.96041	validation_1-rmse:0.99173
[3]	validation_0-rmse:0.93351	validation_1-rmse:0.97589
[4]	validation_0-rmse:0.91499	validation_1-rmse:0.96622
[5]	validation_0-rmse:0.90174	validation_1-rmse:0.95959
[6]	validation_0-rmse:0.89510	validation_1-rmse:0.95747
[7]	validation_0-rmse:0.88987	validation_1-rmse:0.95878
[8]	validation_0-rmse:0.88355	validation_1-rmse:0.95513
[9]	validation_0-rmse:0.87835	validation_1-rmse:0.95395
[10]	validatio

190.4997754096985

In [75]:
X_train_new=(X_train[['item_category_id','item_cnt_month_lag_1','date_item_avg_item_cnt_lag_1','delta_price_lag','date_cat_avg_item_cnt_lag_1','item_id','date_avg_item_cnt_lag_1','month','subtype_code','date_shop_cat_avg_item_cnt_lag_1','shop_id','date_item_avg_item_cnt_lag_2','item_first_sale','item_shop_first_sale','item_cnt_month_lag_2','date_block_num','item_cnt_month_lag_3','date_shop_avg_item_cnt_lag_1']])
X_valid_new=(X_valid[['item_category_id','item_cnt_month_lag_1','date_item_avg_item_cnt_lag_1','delta_price_lag','date_cat_avg_item_cnt_lag_1','item_id','date_avg_item_cnt_lag_1','month','subtype_code','date_shop_cat_avg_item_cnt_lag_1','shop_id','date_item_avg_item_cnt_lag_2','item_first_sale','item_shop_first_sale','item_cnt_month_lag_2','date_block_num','item_cnt_month_lag_3','date_shop_avg_item_cnt_lag_1']])
X_test_new=(X_test[['item_category_id','item_cnt_month_lag_1','date_item_avg_item_cnt_lag_1','delta_price_lag','date_cat_avg_item_cnt_lag_1','item_id','date_avg_item_cnt_lag_1','month','subtype_code','date_shop_cat_avg_item_cnt_lag_1','shop_id','date_item_avg_item_cnt_lag_2','item_first_sale','item_shop_first_sale','item_cnt_month_lag_2','date_block_num','item_cnt_month_lag_3','date_shop_avg_item_cnt_lag_1']])


In [78]:
ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train_new, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train_new, Y_train), (X_valid_new, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

[23:13:06] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:0.97322	validation_1-rmse:0.98745
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:0.89277	validation_1-rmse:0.93537
[2]	validation_0-rmse:0.86140	validation_1-rmse:0.91840
[3]	validation_0-rmse:0.84951	validation_1-rmse:0.91646
[4]	validation_0-rmse:0.84178	validation_1-rmse:0.92022
[5]	validation_0-rmse:0.83604	validation_1-rmse:0.92159
[6]	validation_0-rmse:0.83310	validation_1-rmse:0.92207
[7]	validation_0-rmse:0.82787	validation_1-rmse:0.92033
[8]	validation_0-rmse:0.82435	validation_1-rmse:0.92062
[9]	validation_0-rmse:0.82261	validation_1-rmse:0.92036
[10]	validatio

106.83994936943054

In [82]:
from sklearn.neural_network import MLPRegressor

mlreg = MLPRegressor(random_state=1, max_iter=100,verbose=True)

mlreg.fit(X_train_new, Y_train)


Iteration 1, loss = 91.32896379
Iteration 2, loss = 5.28810295
Iteration 3, loss = 3.99633481
Iteration 4, loss = 3.12092560
Iteration 5, loss = 2.57440929
Iteration 6, loss = 2.24028435
Iteration 7, loss = 1.98975815
Iteration 8, loss = 1.66666246
Iteration 9, loss = 1.33408460
Iteration 10, loss = 1.08962808
Iteration 11, loss = 0.89279114
Iteration 12, loss = 0.76163098
Iteration 13, loss = 0.65095418
Iteration 14, loss = 0.56275764
Iteration 15, loss = 0.51070089
Iteration 16, loss = 0.46811387
Iteration 17, loss = 0.44014630
Iteration 18, loss = 0.43528866
Iteration 19, loss = 0.43421891
Iteration 20, loss = 0.43260115
Iteration 21, loss = 0.43223288
Iteration 22, loss = 0.43121808
Iteration 23, loss = 0.43058389
Iteration 24, loss = 0.42974885
Iteration 25, loss = 0.42970779
Iteration 26, loss = 0.42928281
Iteration 27, loss = 0.42923980
Iteration 28, loss = 0.42901570
Iteration 29, loss = 0.42873520
Iteration 30, loss = 0.42862341
Iteration 31, loss = 0.42853982
Iteration 32, lo

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=100,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [84]:
predict=mlreg.predict(X_valid_new)

In [86]:
import sklearn
sklearn.metrics.mean_squared_error(Y_valid, predict)

0.9354253198488386

In [87]:
mlreg.fit(X_train, Y_train)

Iteration 1, loss = 239.69669121
Iteration 2, loss = 11.16106540
Iteration 3, loss = 8.31519080
Iteration 4, loss = 6.63066506
Iteration 5, loss = 5.09455100
Iteration 6, loss = 4.12104323
Iteration 7, loss = 3.30412524
Iteration 8, loss = 2.84973553
Iteration 9, loss = 2.48650724
Iteration 10, loss = 2.01068248
Iteration 11, loss = 1.70432156
Iteration 12, loss = 1.46982872
Iteration 13, loss = 1.21170858
Iteration 14, loss = 0.97379857
Iteration 15, loss = 0.80499072
Iteration 16, loss = 0.72247260
Iteration 17, loss = 0.65449381
Iteration 18, loss = 0.58005976
Iteration 19, loss = 0.53868114
Iteration 20, loss = 0.49866397
Iteration 21, loss = 0.47162199
Iteration 22, loss = 0.45385379
Iteration 23, loss = 0.43317546
Iteration 24, loss = 0.42151173
Iteration 25, loss = 0.41880608
Iteration 26, loss = 0.41765226
Iteration 27, loss = 0.41631978
Iteration 28, loss = 0.41622206
Iteration 29, loss = 0.41510830
Iteration 30, loss = 0.41481764
Iteration 31, loss = 0.41404701
Iteration 32, 

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=100,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [89]:
predict1=mlreg.predict(X_valid)

In [90]:
sklearn.metrics.mean_squared_error(Y_valid, predict1)

0.909830635340128

In [91]:
predict1_test=mlreg.predict(X_test)

In [92]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": predict1_test
})
submission.to_csv('neural1.csv', index=False)

# Training neural net on whole data

In [93]:

X_train_full = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y_train_full = data[data.date_block_num < 34]['item_cnt_month']
#X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
#Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)


Iteration 1, loss = 91.32896379
Iteration 2, loss = 5.28810295
Iteration 3, loss = 3.99633481
Iteration 4, loss = 3.12092560
Iteration 5, loss = 2.57440929
Iteration 6, loss = 2.24028435
Iteration 7, loss = 1.98975815
Iteration 8, loss = 1.66666246
Iteration 9, loss = 1.33408460
Iteration 10, loss = 1.08962808
Iteration 11, loss = 0.89279114
Iteration 12, loss = 0.76163098
Iteration 13, loss = 0.65095418
Iteration 14, loss = 0.56275764
Iteration 15, loss = 0.51070089
Iteration 16, loss = 0.46811387
Iteration 17, loss = 0.44014630
Iteration 18, loss = 0.43528866
Iteration 19, loss = 0.43421891
Iteration 20, loss = 0.43260115
Iteration 21, loss = 0.43223288
Iteration 22, loss = 0.43121808
Iteration 23, loss = 0.43058389
Iteration 24, loss = 0.42974885
Iteration 25, loss = 0.42970779
Iteration 26, loss = 0.42928281
Iteration 27, loss = 0.42923980
Iteration 28, loss = 0.42901570
Iteration 29, loss = 0.42873520
Iteration 30, loss = 0.42862341
Iteration 31, loss = 0.42853982
Iteration 32, lo

In [94]:
mlreg.fit(X_train_full, Y_train_full)


Iteration 1, loss = 233.48082608
Iteration 2, loss = 10.46957409
Iteration 3, loss = 8.20473445
Iteration 4, loss = 6.27314815
Iteration 5, loss = 4.90326701
Iteration 6, loss = 3.91704205
Iteration 7, loss = 3.20302175
Iteration 8, loss = 2.73845699
Iteration 9, loss = 2.29375822
Iteration 10, loss = 1.87799191
Iteration 11, loss = 1.62913967
Iteration 12, loss = 1.35256302
Iteration 13, loss = 1.10195592
Iteration 14, loss = 0.88131452
Iteration 15, loss = 0.75896515
Iteration 16, loss = 0.68826799
Iteration 17, loss = 0.61390789
Iteration 18, loss = 0.55954205
Iteration 19, loss = 0.51844295
Iteration 20, loss = 0.48839238
Iteration 21, loss = 0.46744418
Iteration 22, loss = 0.44724770
Iteration 23, loss = 0.43374596
Iteration 24, loss = 0.43148827
Iteration 25, loss = 0.42958290
Iteration 26, loss = 0.42807859
Iteration 27, loss = 0.42707760
Iteration 28, loss = 0.42681825
Iteration 29, loss = 0.42614693
Iteration 30, loss = 0.42555226
Iteration 31, loss = 0.42569790
Iteration 32, 

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=100,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [95]:
predict2_test=mlreg.predict(X_test)

In [96]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": predict2_test
})
submission.to_csv('neural2.csv', index=False)

# Stacknet 3

In [97]:
# Level 1 are the base models that take the training dataset as input

l1_clf1 = XGBRegressor( max_depth=8,
                            n_estimators=1000,
                            min_child_weight=300, 
                            colsample_bytree=0.8, 
                            subsample=0.8, 
                            eta=0.3,    
                            seed=42)

l1_clf2 = LGBMRegressor(boosting_type='gbdt',
                        objective="binary",
                        metric="AUC",
                        boost_from_average="false",
                        learning_rate=0.0045,
                        num_leaves=491,
                        max_depth=20,
                        min_child_weight=0.035,
                        feature_fraction=0.38,
                        bagging_fraction=0.42,
                        min_data_in_leaf=100,
                        max_bin=255,
                        importance_type='split',
                        reg_alpha=0.4,
                        reg_lambda=0.65,
                        bagging_seed=seed,
                        random_state=seed,
                        verbosity=-1,
                        subsample=0.85,
                        colsample_bytree=0.8,
                        min_child_samples=79)

l1_clf3 = CatBoostRegressor(learning_rate=0.2,
                            bagging_temperature=0.1, 
                            l2_leaf_reg=30,
                            depth=12, 
                            max_bin=255,
                            iterations=100,
                            loss_function='Logloss',
                            objective='RMSE',
                            eval_metric="AUC",
                            bootstrap_type='Bayesian',
                            random_seed=seed,
                            early_stopping_rounds=10)





# Level 2 models will take predictions from level 1 models as input
# Remember to keep level 2 models smaller
# Basic models like Ridge Regression with large regularization or small random forests work well
l2_clf1 = RandomForestRegressor(n_estimators=250, 
                                max_depth=5, 
                                max_features='sqrt', 
                                random_state=seed)

In [98]:
models = [[l1_clf1, l1_clf2, l1_clf3], # Level 1
          [l2_clf1]] # Level 2
FOLDS=4
METRIC="rmse"
RESTACKING=False
RETRAIN=True
SEED=12345
VERBOSE=1


In [ ]:
from pystacknet.pystacknet import StackNetRegressor

# Specify parameters for stacked model and begin training
model=StackNetRegressor(models, metric=METRIC, folds=FOLDS,
    restacking=RESTACKING, use_retraining=RETRAIN, 
    random_state=SEED, verbose=VERBOSE)

# Fit the entire model tree
model.fit(X_train, Y_train)

====================== Start of Level 0 ======================
Input Dimensionality 34 at Level 0 
3 models included in Level 0 
[17:14:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
